# Bi-Tempered Logistic Loss

# Dependency
1. [Add this Modified BiTempered Logstic loss](https://www.kaggle.com/durbin164/bitempered-logistic-loss-tensorflow-v2) as dataset in your kernel OR Copy pest [this modified code](https://github.com/durbin-164/BiTemperedLogisticLossTensorflow/blob/master/bi_tempered_loss.py) in you kernel. 

* ## Acknowledgement:

1. [TPU Custom training loop ](https://www.kaggle.com/mgornergoogle/custom-training-loop-with-100-flowers-on-tpu)

2. [Bi-Tempered Logistic loss](https://github.com/google/bi-tempered-loss)

3. Special Thanks [@Alexey Pronin](https://www.kaggle.com/graf10a) for helping me to make custom loss function as a class.
4. [This awesome kernel](https://www.kaggle.com/dimitreoliveira/cassava-leaf-disease-tpu-tensorflow-training) help a lot to start training in TPU.

# [Inference](https://www.kaggle.com/durbin164/tpu-bitempered-logistic-loss-inferance)


## Tensorflow Version Update
**Update Tensorflow version both in Kaggle kernel and TPU cluster.**

In [ ]:
!pip install -U tensorflow==2.3.2


In [ ]:
print("update TPU server tensorflow version...")

!pip install cloud-tpu-client
import tensorflow as tf 
from cloud_tpu_client import Client
print(tf.__version__)
Client().configure_tpu_version(tf.__version__, restart_type='ifNeeded')

### Install efficient net

In [ ]:
!pip install efficientnet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import seaborn as sns
import os
import cv2
import re
import math
import datetime
import time
from collections import namedtuple

from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout,\
        Flatten,GlobalAveragePooling2D,BatchNormalization, Activation

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers

import tensorflow.keras.layers as L
import tensorflow.keras.backend as K

from kaggle_datasets import KaggleDatasets

import efficientnet.keras as efn 
print(tf.__version__)

#### Initialize TPU cluster

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:

#setting
SEED = 100
DEBUG = False
WANDB = False
# TARGET_SIZE = 512
VALIDATION_SIZE = 0.2
BATCH_SIZE = 16 *REPLICAS
LEARNING_RATE = 3e-5 * REPLICAS
EPOCHS=2
PATIENT = 10
MODEL_NAME = "EfficentNetB4"
N_FOLDS = 5
PATIENCE = 15  #This is for early stopping.

#For BiTempered loss function
T_1 = 0.6
T_2 = 1.4
SMOOTH_FRACTION = 0.1
N_ITER = 5


HEIGHT = 512
WIDTH = 512
HEIGHT_RS = 512
WIDTH_RS = 512
CHANNELS = 3
N_CLASSES = 5
CLASSES = 5

MODEL_SAVE_PATH = ""

if DEBUG:
    EPOCHS = 2
    # LEARNING_RATE = 3e-5 * REPLICAS
    # TARGET_SIZE = 512
    # BATCH_SIZE = 8*REPLICAS

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r'-([0-9]*)\.').search(filename).group(1)) for filename in filenames]
    return np.sum(n)


original_data = KaggleDatasets().get_gcs_path('cassava-leaf-disease-classification')+'/train_tfrecords' # Original dataset
# original_data = 'gs://kds-3128bc3ed453f4b14d6378d8d9756faab6734c137e2c9c1d6e52bc6e/train_tfrecords'
custom_512_512 = KaggleDatasets().get_gcs_path('cassava-leaf-disease-tfrecords-center-512x512')
external_custom_512_512 = KaggleDatasets().get_gcs_path('cassava-leaf-disease-tfrecords-external-512x512')

print(original_data)
print(custom_512_512)

# GCS_PATH = os.path.join(external_custom_512_512)
# print(GCS_PATH)
# DATASET_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/*.tfrec')

# NUM_FILES = len(DATASET_FILENAMES)
# print("Number of files: ",NUM_FILES)

# display(DATASET_FILENAMES)

In [ ]:

original_files = tf.io.gfile.glob(original_data + '/*.tfrec')
custom_files = tf.io.gfile.glob(custom_512_512 + '/*.tfrec')
external_files = tf.io.gfile.glob(external_custom_512_512 + '/*.tfrec')

DATASET_FILENAMES = custom_files


NUM_FILES = len(DATASET_FILENAMES)
print("Number of files: ",NUM_FILES)

np.random.shuffle(DATASET_FILENAMES)

display(DATASET_FILENAMES)


## Data Augmentation

In [ ]:
# data augmentation @cdeotte kernel: https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96
def transform_rotation(image, height, rotation):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated
    DIM = height
    XDIM = DIM%2 #fix for size 331
    
    rotation = rotation * tf.random.uniform([1],dtype='float32')
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape(tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3])

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(rotation_matrix,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES 
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3])

def transform_shear(image, height, shear):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly sheared
    DIM = height
    XDIM = DIM%2 #fix for size 331
    
    shear = shear * tf.random.uniform([1],dtype='float32')
    shear = math.pi * shear / 180.
        
    # SHEAR MATRIX
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape(tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3])    

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(shear_matrix,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES 
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3])

# CutOut
def data_augment_cutout(image, min_mask_size=(int(HEIGHT * .1), int(HEIGHT * .1)), 
                        max_mask_size=(int(HEIGHT * .125), int(HEIGHT * .125))):
    p_cutout = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    
    if p_cutout > .85: # 10~15 cut outs
        n_cutout = tf.random.uniform([], 10, 15, dtype=tf.int32)
        image = random_cutout(image, HEIGHT, WIDTH, 
                              min_mask_size=min_mask_size, max_mask_size=max_mask_size, k=n_cutout)
    elif p_cutout > .6: # 5~10 cut outs
        n_cutout = tf.random.uniform([], 5, 10, dtype=tf.int32)
        image = random_cutout(image, HEIGHT, WIDTH, 
                              min_mask_size=min_mask_size, max_mask_size=max_mask_size, k=n_cutout)
    elif p_cutout > .25: # 2~5 cut outs
        n_cutout = tf.random.uniform([], 2, 5, dtype=tf.int32)
        image = random_cutout(image, HEIGHT, WIDTH, 
                              min_mask_size=min_mask_size, max_mask_size=max_mask_size, k=n_cutout)
    else: # 1 cut out
        image = random_cutout(image, HEIGHT, WIDTH, 
                              min_mask_size=min_mask_size, max_mask_size=max_mask_size, k=1)

    return image

def random_cutout(image, height, width, channels=3, min_mask_size=(10, 10), max_mask_size=(80, 80), k=1):
    assert height > min_mask_size[0]
    assert width > min_mask_size[1]
    assert height > max_mask_size[0]
    assert width > max_mask_size[1]

    for i in range(k):
      mask_height = tf.random.uniform(shape=[], minval=min_mask_size[0], maxval=max_mask_size[0], dtype=tf.int32)
      mask_width = tf.random.uniform(shape=[], minval=min_mask_size[1], maxval=max_mask_size[1], dtype=tf.int32)

      pad_h = height - mask_height
      pad_top = tf.random.uniform(shape=[], minval=0, maxval=pad_h, dtype=tf.int32)
      pad_bottom = pad_h - pad_top

      pad_w = width - mask_width
      pad_left = tf.random.uniform(shape=[], minval=0, maxval=pad_w, dtype=tf.int32)
      pad_right = pad_w - pad_left

      cutout_area = tf.zeros(shape=[mask_height, mask_width, channels], dtype=tf.uint8)

      cutout_mask = tf.pad([cutout_area], [[0,0],[pad_top, pad_bottom], [pad_left, pad_right], [0,0]], constant_values=1)
      cutout_mask = tf.squeeze(cutout_mask, axis=0)
      image = tf.multiply(tf.cast(image, tf.float32), tf.cast(cutout_mask, tf.float32))

    return image

In [ ]:
def data_augment(image, label):
    p_rotation = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_shear = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_cutout = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    
#     # Shear
#     if p_shear > .2:
#         if p_shear > .6:
#             image = transform_shear(image, HEIGHT, shear=20.)
#         else:
#             image = transform_shear(image, HEIGHT, shear=-20.)
            
    # Rotation
    if p_rotation > .2:
        if p_rotation > .6:
            image = transform_rotation(image, HEIGHT, rotation=45.)
        else:
            image = transform_rotation(image, HEIGHT, rotation=-45.)
            
    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
#     if p_spatial > .75:
#         image = tf.image.transpose(image)
        
#     # Rotates
#     if p_rotate > .75:
#         image = tf.image.rot90(image, k=3) # rotate 270º
#     elif p_rotate > .5:
#         image = tf.image.rot90(image, k=2) # rotate 180º
#     elif p_rotate > .25:
#         image = tf.image.rot90(image, k=1) # rotate 90º
        
    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower=.7, upper=1.3)
    if p_pixel_2 >= .4:
        image = tf.image.random_contrast(image, lower=.8, upper=1.2)
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta=.1)
        
#     # Crops
#     if p_crop > .6:
#         if p_crop > .9:
#             image = tf.image.central_crop(image, central_fraction=.5)
#         elif p_crop > .8:
#             image = tf.image.central_crop(image, central_fraction=.6)
#         elif p_crop > .7:
#             image = tf.image.central_crop(image, central_fraction=.7)
#         else:
#             image = tf.image.central_crop(image, central_fraction=.8)
#     elif p_crop > .3:
#         crop_size = tf.random.uniform([], int(HEIGHT*.6), HEIGHT, dtype=tf.int32)
#         image = tf.image.random_crop(image, size=[crop_size, crop_size, CHANNELS])
            
    image = tf.image.resize(image, size=[HEIGHT, WIDTH])

    if p_cutout > .5:
        image = data_augment_cutout(image)
        
    return image, label

## Dataset prepare

In [ ]:
def one_hot(image, label):
    
    label = tf.one_hot(label, N_CLASSES, dtype = tf.float32)
    
    
    return image,label
    

# Datasets utility functions
def decode_image(image_data):
    """
        1. Decode a JPEG-encoded image to a uint8 tensor.
        2. Cast tensor to float and normalizes (range between 0 and 1).
        3. Resize and reshape images to the expected size.
    """
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float64) / 255.0
                      
    image = tf.image.resize(image, [HEIGHT, WIDTH])
    image = tf.reshape(image, [HEIGHT, WIDTH, 3])
    return image

def read_tfrecord(example, labeled=True):
    """
        1. Parse data based on the 'TFREC_FORMAT' map.
        2. Decode image.
        3. If 'labeled' returns (image, label) if not (image, name).
    """
    if labeled:
        TFREC_FORMAT = {
            'image': tf.io.FixedLenFeature([], tf.string), 
            'target': tf.io.FixedLenFeature([], tf.int64), 
        }
    else:
        TFREC_FORMAT = {
            'image': tf.io.FixedLenFeature([], tf.string), 
            'image_name': tf.io.FixedLenFeature([], tf.string), 
        }
    example = tf.io.parse_single_example(example, TFREC_FORMAT)
    image = decode_image(example['image'])
    if labeled:
#         label_or_name = tf.one_hot(tf.cast(example['target'], tf.int32), N_CLASSES, dtype=tf.float64)
        label_or_name = tf.cast(example['target'], tf.int32)
    else:
        label_or_name =  example['image_name']
    return image, label_or_name

def load_dataset(filenames, labeled=True, ordered=False):
    """
        Create a Tensorflow dataset from TFRecords.
    """
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(lambda x: read_tfrecord(x, labeled=labeled), num_parallel_calls=AUTO)
    return dataset

def get_dataset(FILENAMES, labeled=True, ordered=False, repeated=False, augment=False, drop_remainder=False):
    """
        Return a Tensorflow dataset ready for training or inference.
    """
    dataset = load_dataset(FILENAMES, labeled=labeled, ordered=ordered)
    dataset = dataset.map(one_hot, num_parallel_calls=AUTO)
    if augment:
        dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    if repeated:
        dataset = dataset.repeat()
    if not ordered:
        dataset = dataset.shuffle(SEED)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=drop_remainder)
    dataset = dataset.prefetch(AUTO)
    return dataset

## Bi-Tempered Logistic loss function

### If you add [this Loss function](https://www.kaggle.com/durbin164/bitempered-logistic-loss-tensorflow-v2) as dataset keep next cell otherwise replace next cell with [this link code](https://github.com/durbin-164/BiTemperedLogisticLossTensorflow/blob/master/bi_tempered_loss.py).



In [ ]:
# import module we'll need to import our custom module
from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "../input/bitempered-logistic-loss-tensorflow-v2/bi_tempered_loss.py", dst = "../working/loss.py")

# import all our functions
from loss import bi_tempered_logistic_loss

In [ ]:
with strategy.scope():
  class BiTemperedLogisticLoss(tf.keras.losses.Loss):
    def __init__(self, t1, t2, lbl_smth, n_iter):
      super(BiTemperedLogisticLoss, self).__init__()
      self.t1 = t1
      self.t2 = t2
      self.lbl_smth = lbl_smth
      self.n_iter = n_iter

    def call(self, y_true, y_pred):
      return bi_tempered_logistic_loss(y_pred, y_true, self.t1, self.t2, self.lbl_smth, self.n_iter)

## Prepared Model

In [ ]:
with strategy.scope():
    class LRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
        def __call__(self, step):
            return lrfn(epoch=step//STEPS_PER_EPOCH)
    
    def get_model():

        backbon = efn.EfficientNetB5(weights='noisy-student',
                                     include_top=False,
                                     input_shape=(HEIGHT, WIDTH,3))

        backbon.trainable = True # Full Training

        model = tf.keras.Sequential([
            backbon,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(5, activation='softmax')
        ])


        optimizer = tf.keras.optimizers.Adam()

        T_1 = 0.6
        T_2 = 1.4
        SMOOTH_FRACTION = 0.1
        N_ITER = 5

        model.compile(optimizer=optimizer,
                      loss=BiTemperedLogisticLoss(t1=T_1, t2=T_2, lbl_smth=SMOOTH_FRACTION, n_iter=N_ITER),
                      metrics=['categorical_accuracy'], )


        return model

## Learning rate Scheduler

In [ ]:
LR_START = 0.00001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .8
        

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) 
    return lr

rng = [i for i in range(EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

## Callback function

In [ ]:
def get_checkpoint(model_save_path, is_save_best = True):
    return ModelCheckpoint(model_save_path, 
                             monitor= 'val_loss', 
                             verbose=1, 
                             save_best_only=is_save_best, 
                             mode= 'min', 
                             save_weights_only = False)
    
def get_early_stopping():
    return EarlyStopping(monitor = 'val_loss', min_delta = 0.0001, 
                           patience = PATIENCE, mode = 'min', verbose = 1,
                           restore_best_weights = True)
    

def get_learning_rate_decay():
  return ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, 
                              patience = 1, min_delta = 0.0001, 
                              mode = 'min', verbose = 1)


def get_model_callback( fold_num):
    model_save_path_best = f'{MODEL_NAME}_best_fold_{fold_num}_.h5'
    print("Best model save path: ", model_save_path_best)

    model_save_path_last = f'{MODEL_NAME}_last_fold_{fold_num}_.h5'
    print("Last model save path: ", model_save_path_last)

    checkpoint_best = get_checkpoint(model_save_path_best, is_save_best = True)
    checkpoint_last = get_checkpoint(model_save_path_last, is_save_best = False)

    early_stopping = get_early_stopping()
#     learning_rate_decay = get_learning_rate_decay()
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)
    
#     if WANDB:
#         wandb.run.name= f'{MODEL_NAME}_fold_{fold_num}'
#         [WandbCallback(),checkpoint_best, checkpoint_last, early_stopping]

    return [checkpoint_best, checkpoint_last, early_stopping, lr_callback]

## K-flod and train model

In [ ]:
skf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
oof_pred = []; oof_labels = []; history_list = []


for fold , (X_train, X_valid) in enumerate(skf.split(np.arange(NUM_FILES))):

    print("Start Fold: ",fold);
    # print("X_valid: ", X_valid)
    MODEL_SAVE_PATH = f"{MODEL_NAME}_{fold}.h5"

    if tpu: tf.tpu.experimental.initialize_tpu_system(tpu)

#     TRAIN_FILENAMES = tf.io.gfile.glob([GCS_PATH + '/ld_train%.2i*.tfrec' % x for x in X_train])
#     VALID_FILENAMES = tf.io.gfile.glob([GCS_PATH + '/ld_train%.2i*.tfrec' % x for x in X_valid])
    
    TRAIN_FILENAMES = [DATASET_FILENAMES[x] for x in X_train]
    VALID_FILENAMES = [DATASET_FILENAMES[x] for x in X_valid]
    
    
#     np.random.shuffle(DATASET_FILENAMES)
    
    ct_train = count_data_items(TRAIN_FILENAMES)
    ct_valid = count_data_items(VALID_FILENAMES)

    print("Train File: ",ct_train)
    print("Valid File: ", ct_valid)

    STEPS_PER_EPOCH =  ct_train// BATCH_SIZE
    VALIDATION_STEPS = ct_valid // BATCH_SIZE
    print("Train Step: ",STEPS_PER_EPOCH)
    print("Valid Step: ",VALIDATION_STEPS)


    callback_list = get_model_callback(fold)

    tf.keras.backend.clear_session()
    
    with strategy.scope():
        model = get_model()

        history = model.fit( x=get_dataset(TRAIN_FILENAMES, labeled=True, ordered=False, repeated=True, augment=True,drop_remainder=False),
                                steps_per_epoch = STEPS_PER_EPOCH,
                                epochs = EPOCHS,
                                validation_data = get_dataset(VALID_FILENAMES, labeled=True, ordered=True, repeated=True, augment=False, drop_remainder=False), 
                                validation_steps = VALIDATION_STEPS,
                                callbacks = callback_list
                              )

    history_list.append(history)



    ds_valid = get_dataset(VALID_FILENAMES, labeled=True, ordered=True, repeated=False, augment=False)
    oof_labels.append([target.numpy() for img, target in iter(ds_valid.unbatch())])
    x_oof = ds_valid.map(lambda image, image_name: image)
    oof_pred.append(np.argmax(model.predict(x_oof), axis=-1))
    
    ## RESULTS
    print(f"#### FOLD {fold} OOF Accuracy = {np.max(history.history['val_categorical_accuracy']):.3f}")

In [ ]:
def plot_hist(hist):
    X_epochs = len(hist.history["categorical_accuracy"])
    plt.figure(figsize=(15,5))
    plt.plot(np.arange(X_epochs), hist.history["categorical_accuracy"], '-o', label='Train Accuracy',color='#ff7f0e')
    plt.plot(np.arange(X_epochs), hist.history["val_categorical_accuracy"], '-o',label='Val Accuracy',color='#1f77b4')
    plt.xlabel('Epoch',size=14)
    plt.ylabel('Accuracy',size=14)
    plt.legend(loc=2)
    
    plt2 = plt.gca().twinx()
    plt2.plot(np.arange(X_epochs) ,hist.history['loss'],'-o',label='Train Loss',color='#2ca02c')
    plt2.plot(np.arange(X_epochs) ,hist.history['val_loss'],'-o',label='Val Loss',color='#d62728')
    plt.legend(loc=3)
    plt.ylabel('Loss',size=14)
    plt.title("Model Accuracy and loss")
    
    #plt.legend(["train", "validation"], loc="upper left")
    
    plt.savefig('loss.png')
    plt.show()

for fold, history in enumerate(history_list):
    print(f'\nFOLD: {fold}')
    plot_hist(history) 

In [ ]:
y_true = np.argmax(np.concatenate(oof_labels),-1)
y_preds = np.concatenate(oof_pred)

print(classification_report(y_true, y_preds))

print("****************")
print(confusion_matrix(y_true, y_preds ))